# STATS 607
## Week 4.2: GPU programming

Let's estimate $\pi$:

- $\pi$ = area of the unit circle.
- If we randomly drop a point in the unit square, it will on average fall inside the circle exactly $\pi/4$ of the time.

In [10]:
import numpy as np
rng = np.random.default_rng(1)

In [12]:
def pi_mc(rng):
    x, y = rng.uniform(-1, 1, size=(2,))
    return x ** 2 + y ** 2 <= 1

4 * pi_mc(rng).mean()

4.0

- Monte Carlo converges at rate $n^{-1/2}$. 
- Therefore if we want an extra decimal point of precision, we need to generate about $100\times$ more samples.
- $\therefore$ if we want to (naively) estimate $\pi$ to 8 decimal points, we need about ten quadrillion ($10^{16}$) samples.
- (Exercise: compute the exact number. I get $6.7 \times 10^{15}$.)

## Improving the MC iteration
- MC is a simple calculation that we can run in parallel.
- Additionally, we don't need to store the intermediate results. 
- We can use a running sum/average.

In [11]:
import numba as nb
import random

@nb.jit(nogil=False)
def pi_mc_nb(n):
    s = 0
    for i in range(n):
        x = random.random()
        y = random.random()
        s += x ** 2 + y ** 2 <= 1
    return 4 * s / n

pi_mc_nb(100000000)

3.14137308

## Parallelization

Now let's run in parallel across all processors:

In [39]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

In [42]:
%%time
with ThreadPoolExecutor() as pool:
    results = list(pool.map(pi_mc_nb, [10_000_000] * 24))

CPU times: user 3.31 s, sys: 62.2 ms, total: 3.37 s
Wall time: 3.29 s


## GPU parallelization

There are two ways to interact with GPUs:
1. High level (ufuncs, Jax, etc.)
2. Low level (write CUDA-ish code)

\#1 almost always adequate. However it will not always "max out" the GPU.

In [25]:
@nb.vectorize('int8(float32, float32)', target="cuda")
def pi_mc_nb(x, y):
    return x ** 2 + y ** 2 <= 1

x, y = rng.uniform(size=(2, 1000000000)).astype(np.float32)
4 * pi_mc_nb(x, y).mean()

3.141584608

In [29]:
# Using jax
import jax
jax.config.update('jax_log_compiles', True)

@jax.jit
def pi_mc_jax(x, y):
    return x ** 2 + y ** 2 <= 1

x, y = rng.uniform(size=(2, 1000000000)).astype(np.float32)
4 * pi_mc_nb(x, y).mean()

3.141646408

# CUDA

- Computation in CUDA is organized into grids, blocks and threads.
  - Blocks are groups of threads.
  - Grids are collections of blocks.
  - (Usually there's only one grid.)
- Threads are the smallest unit of execution in CUDA.
  - Each thread executes a kernel function independently.
  - Threads within a block can synchronize with each other and share memory.
  - Blocks cannot synchronize (easily), and can only share global memory.

![threads and blockl](https://upload.wikimedia.org/wikipedia/commons/5/5b/Block-thread.svg)

## Concurrency
- Threads within a block execute concurrently.
- Blocks are scheduled on streaming multiprocessors (SMs) and can run in any order.

## Memory
- Shared Memory: Accessible by all threads within a block. (fast)
- Global Memory: Accessible by all threads across all blocks, but has higher latency. (slow)

In [1]:
from numba import cuda

@cuda.jit
def add_kernel(x, y, out):
    tx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    ty = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_size = cuda.blockDim.x  # number of threads per block
    grid_size = cuda.gridDim.x    # number of blocks in the grid
    
    start = tx + ty * block_size
    stride = block_size * grid_size

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

## Memory management
- The GPU has its own memory. 
- Copying to (from) the system memory is called _device to (from) host transfer_.
- This can take longer than running the computation! So it can pay to manage this yourself.

In [35]:

import numba.cuda as cuda

@cuda.jit
def monte_carlo_pi_kernel(count, rng_states, results):
    idx = cuda.grid(1)
    s = 0.
    n = 2 ** 20
    if idx < count:
        for i in range(n):
            x = cuda.random.xoroshiro128p_uniform_float32(rng_states, idx)
            y = cuda.random.xoroshiro128p_uniform_float32(rng_states, idx)
            s += x**2 + y**2 <= 1.0
        results[idx] = s / (1. * n)

In [7]:
import numba.cuda.random

def estimate_pi(num_samples):
    threads_per_block = 256
    blocks = (num_samples + threads_per_block - 1) // threads_per_block

    rng_states = numba.cuda.random.create_xoroshiro128p_states(num_samples, seed=1)
    results = cuda.device_array(num_samples, dtype=np.float32)

    monte_carlo_pi_kernel[blocks, threads_per_block](num_samples, rng_states, results)
    pi_hat = 4 * results.copy_to_host().mean()
    return pi_hat

estimate_pi(2 ** 20)

3.141592502593994

## Synchronization
- Sometimes you need to synchronize between threads. 
- For example, suppose we changed our kernel to just write a single count per block:

In [16]:
@cuda.jit
def monte_carlo_pi_kernel(rng_states, results):
    idx = cuda.grid(1)
    n = 2 ** 20
    s = 0.
    for i in range(n):
        x = cuda.random.xoroshiro128p_uniform_float32(rng_states, idx)
        y = cuda.random.xoroshiro128p_uniform_float32(rng_states, idx)
        s += x**2 + y**2 <= 1.0
    results[cuda.blockIdx.x, 0] += s
    results[cuda.blockIdx.x, 1] += n

In [18]:
import numba.cuda.random
import numpy as np

def estimate_pi(num_samples):
    threads_per_block = 256
    assert num_samples % 256 == 0
    blocks = num_samples // threads_per_block
    rng_states = numba.cuda.random.create_xoroshiro128p_states(num_samples, seed=1)
    results = cuda.to_device(np.zeros([blocks, 2]))
    monte_carlo_pi_kernel[blocks, threads_per_block](rng_states, results)
    res = results.copy_to_host().sum(0)
    pi_hat = res[:, 0] / res[:, 1]
    return pi_hat

estimate_pi(1024 * 256)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [39]:
blocks

NameError: name 'blocks' is not defined